In [1]:
import torch, clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [2]:
image_path = "data/cropped_clahe/2_visit01_photo01.JPG"
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
with torch.no_grad():
    image_features = model.encode_image(image)

In [3]:
import pandas as pd
severity = pd.read_csv("/data/avramidi/chla_fundus/consensus_grades_severity.csv")

In [1]:
import evaluate
roc_auc_score = evaluate.load("roc_auc", "multiclass")
refs = [1, 0, 1, 2, 2, 0]
pred_scores = [[0.3, 0.5, 0.2],
                 [0.7, 0.2, 0.1],
                 [0.005, 0.99, 0.005],
                 [0.2, 0.3, 0.5],
                 [0.1, 0.1, 0.8],
                 [0.1, 0.7, 0.2]]
results = roc_auc_score.compute(references=refs,
                                prediction_scores=pred_scores,
                                multi_class='ovr')
print(round(results['roc_auc'], 2))

0.85


In [2]:
import numpy as np

def get_results(logger):
    auc, sns, acc, spc = [], [], [], []
    with open(logger, "r") as f:
        for line in f.readlines():
            if line.startswith("AUC"):
                n = line.split(":")[1][1:5]
                auc.append(float(n))
            if line.startswith("Acc"):
                n = line.split(":")[1][1:5]
                acc.append(float(n))
            if line.startswith("Sen"):
                n = line.split(":")[1][1:5]
                sns.append(float(n))
            if line.startswith("Spe"):
                n = line.split(":")[1][1:5]
                spc.append(float(n))
    return auc, acc, sns, spc

def calc_ci(distr, p):
    dmean = np.mean(distr)
    ci_do = np.percentile(distr, p/2)
    ci_up = np.percentile(distr, 100-p/2)
    return "{:.1f}% ({}% CI, {:.1f} to {:.1f})".format(dmean, 100-p, ci_do, ci_up)

In [4]:
auc, acc, sns, spc = get_results("results/logger_23.txt")
print("AUC =", calc_ci(auc, p=5))
print("Acc =", calc_ci(acc, p=5))
print("Sns =", calc_ci(sns, p=5))
print("Spc =", calc_ci(spc, p=5))

AUC = 70.8% (95% CI, 66.0 to 78.4)
Acc = 71.0% (95% CI, 65.2 to 78.4)
Sns = 67.8% (95% CI, 60.5 to 72.6)
Spc = 73.9% (95% CI, 66.6 to 85.7)


In [5]:
auc, acc, sns, spc = get_results("results/logger_24.txt")
print("AUC =", calc_ci(auc, p=5))
print("Acc =", calc_ci(acc, p=5))
print("Sns =", calc_ci(sns, p=5))
print("Spc =", calc_ci(spc, p=5))

AUC = 70.2% (95% CI, 64.9 to 78.3)
Acc = 71.3% (95% CI, 66.5 to 78.4)
Sns = 65.7% (95% CI, 47.0 to 73.2)
Spc = 74.6% (95% CI, 66.3 to 85.6)
